# **Modeling Notebook**

- This Notebook will be used for testing different models which will then be included in the final Vignette report.

We will be testing the following models:
- **Simple CNN**
    - 3 Convolution Layers with 64 filters for the first 2 and 128 for the second,
    - Maxpooling
    - and a Dense Layer with 128 nodes with a dropout of 50%
- **Hyperband Parameter Training**
    - Uses Keras Hyperband Tuning to start with different architectures at lower epochs and slowly removes unpromising setups, and keeps the best ones and trains further
- **Bayesian Optimization**
    - Uses Optuna to efficiently search the hyperparameter space by modeling the objective function and selecting hyperparameters that are expected to improve the model's performance. This approach is particularly useful for optimizing complex models with many hyperparameters.
- Pre-trained Model additional layering
    - on top of convolution layers to improve 
    - by how much does it improve

Considerations:
- Do we need to consider a better data set?
- how can we get better performance?

In [7]:
##### library imports ##### 

# preprocessing and splitting data packages
import os
from Preprocessing import preprocess_data
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import pickle
import matplotlib.pyplot as plt
# modeling data packages
import keras
import tensorflow as tf
import keras_tuner as kt
import kerastuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import backend as K
import optuna # for bayesian optimization

# set random seeds
np.random.seed(42)
tf.random.set_seed(42)
keras.utils.set_random_seed(42)

# Preprocess Data from Preprocessing file

- Pulls 400 Brain Tumor MRI images from each of 4 different classes, including:
    - Glioma
    - Meningioma  
    - Pituitary
    - Normal (no tumor)

- Ensures they are normalizes, resized, and Grayscaled for quicker computation

In [10]:
my_dir = os.getcwd()
print(f"Current working directory: {my_dir}")

# change this accordingly but should work for all users
glioma_path = my_dir[:-15] + "/data/glioma_tumor"
meningioma_path = my_dir[:-15] + "/data/meningioma_tumor"
pituitary_path = my_dir[:-15] + "/data/pituitary_tumor"
normal_path = my_dir[:-15] + "/data/normal_tumor"


# preprocess data
n_samples = 400 # good starting point to train model
seed = 1234 # reproducibility
glioma_data, glioma_labels = preprocess_data(glioma_path, n_samples, seed)
print("processed glioma data")
print(glioma_data.shape)
meningioma_data, meningioma_labels = preprocess_data(meningioma_path, n_samples, seed)
print("processed meningioma data")
print(meningioma_data.shape)
pituitary_data, pituitary_labels = preprocess_data(pituitary_path, n_samples, seed)
print("processed pituitary data")
print(pituitary_data.shape)
normal_data, normal_labels = preprocess_data(normal_path, n_samples, seed)
print("processed normal data")
print(normal_data.shape)
if glioma_data.shape != meningioma_data.shape or pituitary_data.shape != normal_data.shape:
    raise ValueError("All data must have the same shape")
else:
    print("All data have the same shape")

# combine all data and labels
data = np.concatenate([glioma_data, meningioma_data, pituitary_data, normal_data])
labels = np.concatenate([glioma_labels, meningioma_labels, pituitary_labels, normal_labels])
print(f"Combined data and labels shape: {data.shape}, {labels.shape}")

# split data into training and testing and validation sets
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.2, random_state=42)
train_small, val_data, train_small_labels, val_labels = train_test_split(train_data, train_labels, test_size=0.25, random_state=42)

# one hot encode labels
label_encoder = OneHotEncoder(sparse_output=False)
train_labels = label_encoder.fit_transform(np.array(train_labels).reshape(-1, 1))
test_labels = label_encoder.transform(np.array(test_labels).reshape(-1, 1))
train_small_labels = label_encoder.transform(np.array(train_small_labels).reshape(-1, 1))
val_labels = label_encoder.transform(np.array(val_labels).reshape(-1, 1))

Current working directory: /Users/reese/Documents/School/UC Santa Barbra/PSTAT/PSTAT197/vignette-cnn/scripts/drafts
processed glioma data
(400, 64, 64, 1)
processed meningioma data
(400, 64, 64, 1)
processed pituitary data
(399, 64, 64, 1)
processed normal data
(400, 64, 64, 1)


ValueError: All data must have the same shape

# Visually Inspect

In [ ]:
# inverse transform labels
labels = label_encoder.inverse_transform(train_labels)

# show images from training set
for i in range(9):
    plt.subplot(3, 3, i+1)
    plt.imshow(train_data[i], cmap="gray")
    plt.axis('off')
    plt.title((labels[i]))
plt.show()

# Modeling
- ### We first can start with a simple convolution model as a base level to see how well one archictecture will learn on the training data set.
- ### Next, we will apply a hyperparameter tuning technique, called Hyperband tuning, to find the best setup.
     - Hyperband tuning performs a combination of random search and early stopping to efficiently explore hyperparameter configurations. It allocates resources to promising configurations while quickly discarding those that are underperforming, allowing for faster convergence to optimal hyperparameters.

# Simple CNN to Start

### What is a CNN?
A CNN (convolutional neural network) is a type of network that specializes in processing and predicting grid-like data, making it especially useful for image classification. It utilizes automatic detection of spatial hierarchies and patterns with layers of convolutional filters. Unlike the classic feed-forward network, which connects each neuron to every neuron in the next layer, a CNN uses local receptive fields and shared weights in its convolutional layers, which reduces the number of parameters significantly, and improves the efficiency of the model. 

### Model Flow
- In our example, the filters will pickup on different shapes, edges, and other patterns in the image, by sliding multiple 3x3 kernels convolving with the input image.
- After this we, apply a pooling function of either averaging or maximizing which typically halves the size of our convolved feature.
- Then after another round of convolving and maxpool, we flatten it and pass to a Dense neural network of hidden layers
- Finally the output layer is activated by a softmax function to provide us with probabilities of the classes. 

In [ ]:
# Initialize model
model = Sequential([
  Conv2D(filters=64, kernel_size=(3,3), activation='relu', input_shape=(64, 64, 1)),
  MaxPooling2D(2), # reduce dimensionality
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPooling2D(2), # reduce dimensionality
  Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
  MaxPooling2D(2), # reduce dimensionality
  Flatten(), # flatten the data to feed into the dense layers
  Dense(units=128, activation='relu'),
  Dropout(0.5), # prevent overfitting
  Dense(units=4, activation='softmax') # output layer
])

model.compile(optimizer=keras.optimizers.Adam(), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.summary()

# Training the model

In [ ]:
# Train the model # 

# Early stopping will stop training when the validation loss stops improving for a few epochs, preventing overfitting
early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)

history = model.fit(
    train_data,
    train_labels,
    validation_split=0.2, # Convert sparse matrix
    epochs=20,
    batch_size=32, # mini batch size
    callbacks=[early_stopping]
)

# Evaluate model on test set
test_loss, test_accuracy = model.evaluate(test_data, test_labels)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot training and validation loss
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# TODO LATER once decent performance on simple CNN:
HyperParameter Tuning
- We will apply Hyperband Parameter Tuning to find our best set of params
- Then once we have the best set of hyperparams, we will build a final model and train with those parameters on the full training set.

In [ ]:
# building a hyperband model to find the best model architecture
def build_model(hp):
    '''Hyperband function to test different model architectures'''
    # initialize the model
    model = Sequential([
        Conv2D(filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
               kernel_size=hp.Choice('conv_1_kernel', values=[3,5]),
               activation='relu', input_shape=(64, 64, 1)),
        MaxPooling2D(2),
        Conv2D(filters=hp.Int('conv_2_filter', min_value=32, max_value=128, step=16),
               kernel_size=hp.Choice('conv_2_kernel', values=[3,5]),
               activation='relu'),
        MaxPooling2D(2),
        Conv2D(filters=hp.Int('conv_3_filter', min_value=32, max_value=128, step=16),
               kernel_size=3,
               activation='relu'),
        MaxPooling2D(2),
        Flatten(),
        Dense(units=hp.Int('dense_1_units', min_value=64, max_value=256, step=32),
              activation='relu'),
        Dropout(rate=hp.Float('dropout_1', min_value=0.2, max_value=0.5, step=0.1)),
        Dense(4, activation='softmax')
    ])
    # compile the model
    model.compile(
        optimizer=keras.optimizers.Adam(),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model
# initialize tuner
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=30,
    factor=5,
    directory=my_dir[:-15] + "/scripts/drafts/model_trials",  # Use temporary directory
    project_name='hp_tuning',
    overwrite=False
)
# search for the best hyperparameters
early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
tuner.search(
    train_small, train_small_labels,
    epochs=5,
    validation_data=(val_data, val_labels),
    callbacks=[early_stopping],
    verbose=1
)
# Get the best hyperparameters directly
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Best Hyperparameters:")
print(best_hp.values)
# build the best model
final_model = tuner.hypermodel.build(best_hp)
final_model.compile(optimizer=keras.optimizers.Adam(), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])
final_model.summary()

Once we have our best model, we can then build using the best architecture and train on the full data set.

In [ ]:
# early stopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
history = final_model.fit(train_data, train_labels, 
                epochs=50,
                validation_split=0.15,  
                batch_size=32,
                callbacks=[early_stopping])

# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# evaluate the best model on the test set
test_loss, test_accuracy = final_model.evaluate(test_data, test_labels)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

### Bayesian Approach

In [ ]:
def objective(trial):
    # define hyperparameters to tune
    filters = [trial.suggest_int(f'filters_layer_{i+1}', 32, 128, step=16) for i in range(3)]
    kernel_size = trial.suggest_int('kernel_size', 3, 5)
    dense_units = trial.suggest_int('dense_units', 64, 256, step=32)
    dropout_rate = trial.suggest_float('dropout_rate', 0.2, 0.5, step=0.1)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True)
    early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
    # init model
    model = Sequential()
    model.add(Conv2D(filters=filters[0], 
                     kernel_size=kernel_size, 
                     activation='relu', input_shape=(64, 64, 1))) # specify input for first layer
    model.add(MaxPooling2D(2))
    for i in range(1, 3):
        model.add(Conv2D(filters=filters[i], kernel_size=kernel_size, activation='relu'))
        model.add(MaxPooling2D(2))
    model.add(Flatten())
    model.add(Dense(dense_units, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(4, activation='softmax')) # 4 classes

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    # train the model
    model.fit(train_small, train_small_labels, 
              epochs=20, # fewer epochs for faster training
              validation_data=(val_data, val_labels), 
              callbacks=[early_stopping],
              verbose=0)
    # return the objective: accuracy
    return model.evaluate(test_data, test_labels, verbose=1)[1] 

# Initialize study
Bayes_study = optuna.create_study(direction='maximize')
Bayes_study.optimize(objective, n_trials=50)

In [ ]:
# Print the best hyperparameters and the best accuracy
print("Best Hyperparameters:")
print(Bayes_study.best_params)
print("Best Accuracy:")
print(Bayes_study.best_value)

# Extract data from the study
trials = Bayes_study.trials
values = [trial.value for trial in trials]
trial_numbers = [trial.number for trial in trials]

# Plot optimization history
plt.figure(figsize=(10, 6))
plt.plot(trial_numbers, values, marker='o')
plt.xlabel('Trial Number')
plt.ylabel('Validation Accuracy')
plt.title('Optimization History')
plt.show()

In [ ]:
# build best bayesian model

best_bayes_model = Bayes_study.best_params
bayes_hp = kt.HyperParameters()

for key, value in best_bayes_model.items(): # iterate through the best hyperparameters and set them
    bayes_hp.Fixed(key, value)

bayes_model = tuner.hypermodel.build(bayes_hp)
bayes_model.compile(optimizer=keras.optimizers.Adam(learning_rate=best_bayes_model['learning_rate']), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# train best bayesian model
early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True) # callback to prevent overfitting
bayes_history = bayes_model.fit(train_data, train_labels, 
                          epochs=50, validation_split=0.2, 
                          batch_size=32, callbacks=[early_stopping])

# evaluate best bayesian model
bayes_test_loss, bayes_test_accuracy = bayes_model.evaluate(test_data, test_labels)
print(f"Test Loss: {bayes_test_loss}")
print(f"Test Accuracy: {bayes_test_accuracy}")

## Comparing Bayesian versus Hyperband Tuning

In [ ]:
# plot the accuracy and loss of the two models and the test accuracy of the two models
fig, ax = plt.subplots(2, 1, figsize=(10, 8))
ax[0].plot(history.history['accuracy'], label='HB Train Accuracy')
ax[0].plot(history.history['val_accuracy'], label='HB Validation Accuracy')
ax[0].plot(bayes_history.history['accuracy'], label='Bayes Train Accuracy')
ax[0].plot(bayes_history.history['val_accuracy'], label='Bayes Validation Accuracy')
ax[0].set_title('Model Accuracy')
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('Accuracy')
ax[0].legend()

bars = ax[1].bar(['Bayes', 'Hyperband'], 
                 [bayes_test_accuracy, test_accuracy], 
                 color = ['blue', 'green'], 
                 label = ['Bayes', 'Hyperband'])
for bar in bars:
    height = bar.get_height()
    ax[1].text(bar.get_x() + bar.get_width() / 2, height, f'{height:.4f}', ha='center', va='bottom')
ax[1].set_title('Test Accuracy')
ax[1].set_xlabel('Model')
ax[1].set_ylabel('Accuracy')
ax[1].legend()
plt.tight_layout()
plt.show()